In [ ]:
import tensorflow as tf

import numpy as np
import os
import time
import glob
import string
import random
import time
from keras.layers import Bidirectional
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive



In [ ]:
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
######################### DO NOT RUN THIS CELL!!!!!!!!!!! ##########################
path = '/content/drive/My Drive/COMP 550 Project/data/politics/'
all_files = glob.glob(os.path.join(path, "*.txt"))
print(all_files)
output_path = os.path.join('/content/drive/My Drive/COMP 550 Project/data/', 'politics.txt')
for file in all_files:  
  txt_path = os.path.join(path, file)
  text = open(txt_path, 'rb').read().decode(encoding='utf-8')
  f = open(output_path, "a")
  f.write(text)


['/content/drive/My Drive/COMP 550 Project/data/politics/270.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/158.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/252.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/176.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/041.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/248.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/269.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/019.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/004.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/275.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/116.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/195.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/235.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/140.txt', '/content/drive/My Drive/COMP 550 Project/data/politics/201.txt', '/content

## Load Text & Data Preprocess

In [ ]:
output_path = os.path.join('/content/drive/My Drive/COMP 550 Project/data/', 'politics.txt')
text = open(output_path, 'rb').read() 
text = text.decode(encoding='utf-8')
print(len(text))

1119265


In [ ]:
# ref: https://www.tensorflow.org/tutorials/text/text_generation
seq_length = 100
batch_size = 64
buffer_size = 1200000

def map_func(seq):
    input_text = seq[:-1]
    target_text = seq[1:]
    return input_text, target_text

vocab = sorted(set(text))
vocab_idx = np.array(vocab)

# Assign an integer for each character 
char_to_idx = dict((c, i) for i, c in enumerate(vocab))
text_int = [char_to_idx[c] for c in text]
text_int = np.array(text_int)

data_slice = tf.data.Dataset.from_tensor_slices(text_int)
sequences = data_slice.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(map_func)
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(list(dataset.as_numpy_iterator())[10])
print("Number of batches: ", len(dataset))

(array([[77,  1, 62, ..., 77, 67, 61],
       [77, 70, 67, ..., 72, 67, 78],
       [ 1, 66, 67, ...,  1, 59, 78],
       ...,
       [65,  1, 59, ...,  1, 70, 59],
       [71, 59, 67, ..., 78,  1, 78],
       [73, 78,  1, ..., 67, 72, 65]]), array([[ 1, 62, 73, ..., 67, 61,  1],
       [70, 67, 71, ..., 67, 78, 83],
       [66, 67, 77, ..., 59, 78,  1],
       ...,
       [ 1, 59,  1, ..., 70, 59, 81],
       [59, 67, 72, ...,  1, 78, 66],
       [78,  1, 73, ..., 72, 65, 14]]))
Number of batches:  173


## Model Definitions
** Note: Only run one of these cells

### LSTM Model

In [ ]:
epochs = 50
output_dim = 300
lstm_units = 512
max_len = len(vocab)
model_name = 'LSTM'
checkpoint_path = '/content/drive/My Drive/COMP 550 Project/char models/LSTM/'

def lstm_model(max_len, output_dim, lstm_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_len, output_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(lstm_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(lstm_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(max_len)
    ])
    return model

model = lstm_model(
    max_len=max_len,
    output_dim=output_dim,
    lstm_units=lstm_units,
    batch_size=batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 300)           25800     
_________________________________________________________________
lstm (LSTM)                  (64, None, 512)           1665024   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 512)           2099200   
_________________________________________________________________
dense (Dense)                (64, None, 86)            44118     
Total params: 3,834,142
Trainable params: 3,834,142
Non-trainable params: 0
_________________________________________________________________


### GRU Model

In [ ]:
epochs = 50
output_dim = 300
gru_units = 512
max_len = len(vocab)
model_name = 'GRU'
checkpoint_path = '/content/drive/My Drive/COMP 550 Project/char models/GRU/'

def gru_model(max_len, output_dim, gru_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_len, output_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(gru_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(max_len)
    ])
    return model

model = gru_model(
    max_len=max_len,
    output_dim=output_dim,
    gru_units=gru_units,
    batch_size=batch_size) 

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 300)           25800     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 512)           1250304   
_________________________________________________________________
dense_3 (Dense)              (64, None, 86)            44118     
Total params: 1,320,222
Trainable params: 1,320,222
Non-trainable params: 0
_________________________________________________________________


### Bi-directional LSTM Model

In [ ]:
epochs = 50
output_dim = 300
lstm_units = 512
n_timesteps = 10
max_len = len(vocab)
model_name = 'Bi-LSTM'
checkpoint_path = '/content/drive/My Drive/COMP 550 Project/char models/Bi-LSTM/'

def bi_lstm_model(max_len, output_dim, lstm_units, batch_size, n_timesteps):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_len, output_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                    input_shape=(n_timesteps, 1)),
        tf.keras.layers.Dropout(0.5), 
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                    input_shape=(n_timesteps, 1)),
        tf.keras.layers.Dropout(0.5), 
        tf.keras.layers.Dense(max_len)
    ])
    return model

model = bi_lstm_model(
    max_len=max_len,
    output_dim=output_dim,
    lstm_units=lstm_units,
    batch_size=batch_size,
    n_timesteps=n_timesteps)

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 300)           25800     
_________________________________________________________________
bidirectional_2 (Bidirection (64, None, 1024)          3330048   
_________________________________________________________________
dropout_2 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (64, None, 512)           2623488   
_________________________________________________________________
dropout_3 (Dropout)          (64, None, 512)           0         
_________________________________________________________________
dense_6 (Dense)              (64, None, 86)            44118     
Total params: 6,023,454
Trainable params: 6,023,454
Non-trainable params: 0
____________________________________________

## Start Training 

In [ ]:
def loss(labels, logits):
  loss_output = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return loss_output

model.compile(optimizer='adam', loss=loss)

save_fre = len(dataset) * 3  # Save per 3 epochs
file_path = os.path.join(checkpoint_path, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=file_path, save_weights_only=True, save_freq=save_fre)
history = model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])


Epoch 1/50
173/173 [==============================] - 19s 107ms/step - loss: 1.3920
Epoch 2/50
173/173 [==============================] - 18s 105ms/step - loss: 0.1092
Epoch 3/50
173/173 [==============================] - 18s 105ms/step - loss: 0.0899
Epoch 4/50
173/173 [==============================] - 18s 103ms/step - loss: 0.0806
Epoch 5/50
173/173 [==============================] - 18s 105ms/step - loss: 0.0738
Epoch 6/50
173/173 [==============================] - 19s 107ms/step - loss: 0.0689
Epoch 7/50
173/173 [==============================] - 18s 103ms/step - loss: 0.0641
Epoch 8/50
173/173 [==============================] - 18s 104ms/step - loss: 0.0598
Epoch 9/50
173/173 [==============================] - 19s 107ms/step - loss: 0.0562
Epoch 10/50
173/173 [==============================] - 18s 104ms/step - loss: 0.0528
Epoch 11/50
173/173 [==============================] - 18s 104ms/step - loss: 0.0493
Epoch 12/50
173/173 [==============================] - 19s 107ms/step - lo

## Experiments

In [ ]:
def generate_text(model, start_word, num_sentences, temperature, timeout_period):
    timeout = time.time() + timeout_period
    input_eval = [char_to_idx[s] for s in start_word]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    sentence_count = 0
    while sentence_count < num_sentences and time.time() < timeout: 
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predict = tf.random.categorical(predictions, num_samples=1)
        predicted_id = predict[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        char_output = vocab_idx[predicted_id]
        text_generated.append(char_output)
        if char_output in ['.', '?', '!']:
          sentence_count += 1

    return (start_word + ''.join(text_generated))


### LSTM

In [ ]:
batch_size = 1
output_dim = 300
lstm_units = 512
max_len = len(vocab)
input_tensor = tf.TensorShape([batch_size, None])

model = lstm_model(max_len, output_dim, lstm_units, batch_size=batch_size)
model.load_weights(tf.train.latest_checkpoint(checkpoint_path))
print(tf.train.latest_checkpoint(checkpoint_path))
model.build(input_tensor)
model.summary()

/content/drive/My Drive/COMP 550 Project/char models/LSTM/ckpt_48
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 300)            25800     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 512)            1665024   
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 512)            2099200   
_________________________________________________________________
dense_1 (Dense)              (1, None, 86)             44118     
Total params: 3,834,142
Trainable params: 3,834,142
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_sentences = 5
timeout_period = 60
temperature_list = np.arange(1.0, 0.2, -0.2)
vocab_list = [i for i in vocab if i not in ['?','\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '+', ',', '-', '/', ':', ';', '=', '?', '[', ']', '`', ]]
print(vocab_list)
for temperature in temperature_list:
  start_word = random.choice(vocab_list) 
  print("------ Temperature ", temperature, "------")

  print("Start word : ", start_word)
  new_text = generate_text(model, start_word, num_sentences, temperature, timeout_period)
  print(new_text)


['.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£']
------ Temperature  1.0 ------
Start word :  P
P/9[58ZJE6++5QQEMZ£O?Z55O5QOQOMQOQOM9Q5ZO+JOO5$5VP$$+$6=ZZ3O++O3YV5Ou5X$[XOZ=Z3#£&ZYQ5YY6VV5EVVYDZEVZ5V%Z+Q+V$YJ$[VZXZYYZZ5ZYZZD[ZE+Yv3+XXZX]$=[?RZXV$DXZZXZ+]GYZ#DU$B6YZZZ5YDYV%BV+Z$P]X0&ZZ$YZ9XY]BYV5[ZZZ#XV+OZZ$UY5EVY`YOQZZ$ZJY$5VV`Y5QO[YZZ556OZD$O7VX6ZZQA$ZXZY$+Z[ZP9`ZZM%XYX5%ZK$%XZZXI+YZ3]BZD=q]£Z5PQYZVQZ(YZ[Q[$]ZV$#£VQZYZ5OZZ1[jVXVV$+5BZ6YZV+£ZQXYXZ+Z+#YGVX#$YXYPXZ55BXQYL]8XZV$ZX/VZY[]ZZZY#DDBZJXX[YXY#Z]VEZZ]=ZZ#Z+XZ$$XYVY1XZ7[ZZ#Z$[ZOX/XVYM$!3[=ZZ[+3YX#YYZY$XY5VY$qE$6ZDZYZZJYE5#5XZZ%YVZYZPYC$5E$5YDZO9+VNEZ3X%ZZZX3P$DYEEZZ5+PVYMQZVXZXZ%$[Y£Z5Y#ZZ2PVXXZ]VZ`YQ&&ZQZQ$QZZ£PV[Y$5XZ$XZ5+Z9$$Z]5YYVmZ[`+ZV3+ZV6QD9=D#+ZYQYVZ5DPY#

### GRU

In [ ]:
batch_size = 1
output_dim = 300
gru_units = 512
max_len = len(vocab)
input_tensor = tf.TensorShape([batch_size, None])

model = gru_model(max_len, output_dim, gru_units, batch_size=batch_size)
model.load_weights(tf.train.latest_checkpoint(checkpoint_path))
model.build(input_tensor)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 300)            25800     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 512)            1250304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 86)             44118     
Total params: 1,320,222
Trainable params: 1,320,222
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_sentences = 5
temperature_list = np.arange(1, 0.2, -0.2)
timeout_period = 60  ## terminate the function after 60s if it does not generate sentences with '.', '!', '?'
vocab_list = [i for i in vocab if i not in ['?','\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', ':', ';', '=', '?', '[', ']', '`', ]]
print(vocab_list)
print(model.summary())
for temperature in temperature_list:
  start_word = random.choice(vocab_list) 
  print("------ Temperature ", temperature, "------")

  print("Start word : ", start_word)
  new_text = generate_text(model, start_word, num_sentences, temperature, timeout_period)
  print(new_text)


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£']
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 300)            25800     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 512)            1250304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 86)             44118     
Total params: 1,320,222
Trainable params: 1,320,222
Non-trainable params: 0
_________________________________________________________________
None
------ Temperature  1.0 ------
Start word :  L
Le is 

### Bi-LSTM

In [ ]:
batch_size = 1
output_dim = 300
lstm_units = 512
n_timesteps=10
max_len = len(vocab)
input_tensor = tf.TensorShape([batch_size, None])

model = bi_lstm_model(max_len, output_dim, lstm_units, batch_size=batch_size, n_timesteps=n_timesteps)
model.load_weights(tf.train.latest_checkpoint(checkpoint_path))
print(tf.train.latest_checkpoint(checkpoint_path))
model.build(input_tensor)
model.summary()

/content/drive/My Drive/COMP 550 Project/char models/Bi-LSTM/ckpt_48
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 300)            25800     
_________________________________________________________________
bidirectional_4 (Bidirection (1, None, 1024)           3330048   
_________________________________________________________________
dropout_4 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (1, None, 512)            2623488   
_________________________________________________________________
dropout_5 (Dropout)          (1, None, 512)            0         
_________________________________________________________________
dense_7 (Dense)              (1, None, 86)             44118     
Total params: 6,023,454
Trainable params: 6,023,454

In [ ]:
num_sentences = 5
temperature_list = np.arange(1.0, 0.2, -0.2)
timeout_period = 60
vocab_list = [i for i in vocab if i not in ['?','\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', ':', ';', '=', '?', '[', ']', '`', ]]
print(vocab_list)
for temperature in temperature_list:
  start_word = ' ' # random.choice(vocab_list) 
  print("------ Temperature ", temperature, "------")

  print("Start word : ", start_word)
  new_text = generate_text(model, start_word, num_sentences, temperature, timeout_period)
  print(new_text)


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£']
------ Temperature  1.0 ------
Start word :   
 unhipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipipi

## Generate Text

In [ ]:
start_word = ' '#random.choice(vocab_list) 
print(start_word)
num_sentences = 100
temperature = 1.0
filename = 'bilstm_text.txt'
time_period = 60*3


new_text = generate_text(model, start_word, num_sentences, temperature, time_period)
print(new_text)

output_path = os.path.join('/content/drive/My Drive/COMP 550 Project/char-lstm output/', filename)
f = open(output_path, "a")
f.write(new_text)

 
 cleaflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflflfififififififififificicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicicic

16226